In [9]:
import docker
from docker.models.containers import Container

In [10]:
client = docker.from_env()
containers: list[Container] = client.containers.list()
containers

[<Container: 331b2565616b>,
 <Container: d80e7ec10d30>,
 <Container: c4f8aae5c4ae>,
 <Container: a3153ae8b4a2>]

In [34]:
hdfs_prefix = ['hdfs', 'getconf', '-confKey']
hadoop_prefix = ['hdfs', 'getconf', '-confKey']
yarn_prefix = ['hdfs', 'getconf', '-confKey']

hdfs_attr = ['fs.defaultFS', 'hadoop.tmp.dir', 'io.file.buffer.size', 'dfs.replication', 'dfs.blocksize','dfs.namenode.name.dir']
hadoop_attr = ['mapreduce.framework.name', 'mapreduce.job.tracker', 'mapreduce.task.io.sort.mb']
yarn_attr = ['yarn.resourcemanager.address', 'yarn.nodemanager.resource.memory-mb', 'yarn.scheduler.minimum-allocation-mb']

def checkAttr(container:Container, command_prefix:list, attr_list:list):
    for attr in attr_list:
        cmd = command_prefix + [attr]
        code, output = container.exec_run(cmd)
        msg = output.decode('utf-8').strip()
        if code == 0:
            print(f'{cmd} -> {msg}')
        else:
            print(f'검사 실패: {code}')
            print(f'메시지: {msg}')
    return


In [36]:

for container in containers:
    print(container.name)
    if container.name == 'master':
        checkAttr(container, hdfs_prefix, hdfs_attr)
        checkAttr(container, hadoop_prefix, hadoop_attr)
        checkAttr(container, yarn_prefix, yarn_attr)

worker-3
worker-2
master
['hdfs', 'getconf', '-confKey', 'fs.defaultFS'] -> hdfs://master:9000
['hdfs', 'getconf', '-confKey', 'hadoop.tmp.dir'] -> /tmp/hadoop-root
['hdfs', 'getconf', '-confKey', 'io.file.buffer.size'] -> 4096
['hdfs', 'getconf', '-confKey', 'dfs.replication'] -> 3
['hdfs', 'getconf', '-confKey', 'dfs.blocksize'] -> 134217728
['hdfs', 'getconf', '-confKey', 'dfs.namenode.name.dir'] -> file:///usr/local/hadoop/hdfs/namenode
['hdfs', 'getconf', '-confKey', 'mapreduce.framework.name'] -> yarn
검사 실패: 255
메시지: Configuration mapreduce.job.tracker is missing.
['hdfs', 'getconf', '-confKey', 'mapreduce.task.io.sort.mb'] -> 100
['hdfs', 'getconf', '-confKey', 'yarn.resourcemanager.address'] -> master:8032
['hdfs', 'getconf', '-confKey', 'yarn.nodemanager.resource.memory-mb'] -> -1
['hdfs', 'getconf', '-confKey', 'yarn.scheduler.minimum-allocation-mb'] -> 1024
worker-1
